<a href="https://colab.research.google.com/github/harishpilla/Research/blob/Research/CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary packages
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torch import nn
from torch.utils.data import Dataset
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

In [2]:
random_seed = 1

In [3]:
cifar_training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

cifar_test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(cifar_training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(cifar_test_data, batch_size=32, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [12]:
class ConvNet(nn.Module):
    def __init__(self):
      super(ConvNet, self).__init__()

      self.conv1 = nn.Conv2d(3, 128, kernel_size = 3, stride = 1, padding=1)
      self.conv2 = nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding=1)
      self.conv3 = nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding=1)
      self.conv4 = nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding=1)
      
      self.maxPool = nn.MaxPool2d(2,2)
      self.fc1 = nn.Linear(128*2*2, 256)
      self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
      x = self.maxPool(F.relu(self.conv1(x)))
      x = self.maxPool(F.relu(self.conv2(x)))
      x = self.maxPool(F.relu(self.conv3(x)))
      x = self.maxPool(F.relu(self.conv4(x)))
      x = x.view(x.shape[0],-1)
      x = self.fc1(x)
      x = self.fc2(x)
      output = F.log_softmax(x, dim=1)
      #print(out.shape)
      return output

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    for batch, (input, output) in enumerate(dataloader):
        pred = model(input)
        loss = loss_fn(pred, output)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%100 == 0:
            loss, current = loss.item(), batch * len(input)
            print(f"loss: {loss:>7f} Batch:{batch} [{current:>5d}/{len(dataloader.dataset):>5d}]")


def test(dataloader, model, loss_fn):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch, (input, output) in enumerate(dataloader):
            pred = model(input)
            test_loss += loss_fn(pred, output).item()
            correct += (pred.argmax(1) == output).type(torch.float).sum().item()
    test_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    print(f"Test Error: \nAccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
conv_model = ConvNet()
optimizer = torch.optim.Adam(conv_model.parameters(), lr=3e-4)
loss_fn = nn.CrossEntropyLoss()

In [15]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n------------")
    train(train_dataloader,conv_model, loss_fn, optimizer)
    test(test_dataloader,conv_model, loss_fn)

Epoch 1
------------
loss: 2.303619 Batch:0 [    0/50000]
loss: 2.024759 Batch:100 [ 3200/50000]
loss: 2.020213 Batch:200 [ 6400/50000]
loss: 1.885957 Batch:300 [ 9600/50000]
loss: 1.504452 Batch:400 [12800/50000]
loss: 1.878714 Batch:500 [16000/50000]
loss: 1.509202 Batch:600 [19200/50000]
loss: 1.478866 Batch:700 [22400/50000]
loss: 1.634707 Batch:800 [25600/50000]
loss: 1.448206 Batch:900 [28800/50000]
loss: 1.030814 Batch:1000 [32000/50000]
loss: 1.260387 Batch:1100 [35200/50000]
loss: 1.407252 Batch:1200 [38400/50000]
loss: 1.506083 Batch:1300 [41600/50000]
loss: 1.534229 Batch:1400 [44800/50000]
loss: 0.852032 Batch:1500 [48000/50000]
Test Error: 
 Accuracy: 57.0%, Avg loss: 1.212549 

Epoch 2
------------
loss: 1.100632 Batch:0 [    0/50000]
loss: 1.350000 Batch:100 [ 3200/50000]
loss: 1.344119 Batch:200 [ 6400/50000]
loss: 1.324994 Batch:300 [ 9600/50000]
loss: 0.899782 Batch:400 [12800/50000]
loss: 1.401847 Batch:500 [16000/50000]
loss: 1.034438 Batch:600 [19200/50000]
loss: 1